# Advent of Code 2025 - Day 11

In [1]:
# Test data
test_part1 = """
aaa: hhh
you: bbb ccc
bbb: ddd eee
ccc: ddd eee fff
ddd: ggg
eee: out
fff: out
ggg: out
hhh: ccc fff iii
iii: out
"""

test_part2 = """
svr: aaa bbb
aaa: fft
fft: ccc
bbb: tty
tty: ccc
ccc: ddd eee
ddd: hub
hub: fff
eee: dac
dac: fff
fff: ggg hhh
ggg: out
hhh: out
"""

def parse_graph(data):
    """Parse graph from input data."""
    graph = {}
    for line in data.strip().splitlines():
        left, right = line.split(":")
        node = left.strip()
        outputs = right.strip().split()
        graph[node] = outputs
    return graph

test_graph_part1 = parse_graph(test_part1)
test_graph_part2 = parse_graph(test_part2)

print(f"Test Part 1 nodes: {len(test_graph_part1)}")
print(f"Test Part 2 nodes: {len(test_graph_part2)}")

Test Part 1 nodes: 10
Test Part 2 nodes: 13


## Part 1

In [2]:
def count_paths(graph, start, end):
    """
    Count all paths between two nodes using DFS.
    Avoids cycles by tracking visited nodes in current path.
    """
    def dfs(node, end, visited):
        # If we hit the target → count 1 path
        if node == end:
            return 1

        total = 0
        for nxt in graph.get(node, []):
            # Avoid infinite loops if cycles exist
            if nxt not in visited:
                total += dfs(nxt, end, visited | {nxt})
        return total

    return dfs(start, end, {start})

# Test Part 1
result = count_paths(test_graph_part1, "you", "out")
print(f"Test result: {result}")

Test result: 5


In [3]:
# Load real input
with open("input.txt", "r") as file:
    data = file.read().strip()

real_graph = parse_graph(data)
print(f"Real input nodes: {len(real_graph)}")

Real input nodes: 576


In [4]:
# Part 1 - Real input
result = count_paths(real_graph, "you", "out")
print(f"Part 1 result: {result}")
result

Part 1 result: 652


652

## Part 2

In [5]:
def count_paths_with_required(graph, start, end, required_nodes):
    """
    Count paths from start to end that visit all required_nodes.
    Uses memoization on (node, visited_mask) for efficiency.
    """
    from functools import lru_cache

    required_set = frozenset(required_nodes)
    required_list = sorted(required_nodes)
    num_required = len(required_list)
    MAX_DEPTH = 200  # Reasonable limit to prevent infinite loops

    # Memoized counting when all required nodes visited
    @lru_cache(maxsize=None)
    def count_to_end_memo(node, recent_nodes_tuple):
        """Count paths to end, avoiding recent nodes."""
        recent = set(recent_nodes_tuple)
        if node == end:
            return 1
        total = 0
        for nxt in graph.get(node, []):
            if nxt not in recent:
                new_recent = list(recent_nodes_tuple) + [nxt]
                # Keep last 15 nodes for cycle detection
                if len(new_recent) > 15:
                    new_recent = new_recent[-15:]
                total += count_to_end_memo(nxt, tuple(new_recent))
        return total

    # Main memoization: (node, visited_mask) -> count
    memo = {}
    hits = [0]
    misses = [0]

    def dfs(node, visited_mask, recent_nodes_tuple, depth=0):
        # Depth limit
        if depth > MAX_DEPTH:
            return 0

        # All required visited - use optimized counting
        if all(visited_mask):
            return count_to_end_memo(node, recent_nodes_tuple)

        if node == end:
            return 0

        # Memoization key: only (node, visited_mask) - maximizes cache hits!
        memo_key = (node, visited_mask)
        if memo_key in memo:
            hits[0] += 1
            return memo[memo_key]

        misses[0] += 1

        recent_nodes = list(recent_nodes_tuple)
        # Keep only last 12 nodes for cycle detection
        if len(recent_nodes) > 12:
            recent_nodes = recent_nodes[-12:]
        recent_set = set(recent_nodes)

        total = 0
        for nxt in graph.get(node, []):
            # Cycle check: don't revisit nodes in recent path
            if nxt in recent_set:
                continue

            # Update visited mask
            new_mask = list(visited_mask)
            if nxt in required_set:
                idx = required_list.index(nxt)
                new_mask[idx] = True

            # Update recent nodes
            new_recent = list(recent_nodes) + [nxt]
            if len(new_recent) > 12:
                new_recent = new_recent[-12:]

            total += dfs(nxt, tuple(new_mask), tuple(new_recent), depth + 1)

        memo[memo_key] = total
        return total

    result = dfs(start, tuple([False] * num_required), (start,), 0)
    return result

# Test Part 2
result = count_paths_with_required(test_graph_part2, "svr", "out", ["dac", "fft"])
print(f"Test result: {result}")

Test result: 2


In [6]:
# Part 2 - Real input
result = count_paths_with_required(real_graph, "svr", "out", ["dac", "fft"])
print(f"Part 2 result: {result}")
result

Part 2 result: 362956369749210


362956369749210